https://cloud.google.com/ml-engine/docs/tensorflow/deploying-models

# Prerequisites

* After training a custom estimator, the model is saved in an exportable format using `tf.estimator.Estimator.export_saved_model()` method (see "Tensorflow Estimator/Custom Estimator.ipynb" for more details).
* SavedModel directory structure:
![SegmentLocal](resources/1.png)
* Model inputs and outputs:

`The given SavedModel SignatureDef contains the following input(s):
  inputs['PetalLength'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder_2:0
  inputs['PetalWidth'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder_3:0
  inputs['SepalLength'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder:0
  inputs['SepalWidth'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder_1:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['classes'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: ArgMax:0
  outputs['probabilities'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 3)
      name: Softmax:0
Method name is: tensorflow/serving/predict`

# Overview

* Upload your saved model to a Cloud Storage bucket.
* Create an AI Platform model resource.
* Create an AI Platform version resource, specifying the Cloud Storage path to your saved model.

# Upload model to Cloud Storage bucket

* Set up a GCP project
  * Go to Google Cloud console (https://console.cloud.google.com)
  * Select or create a new project. For example, `Custom Estimator`
  * Select navigation menu, choose <b>Artificial Intelligence/AI Platform</b> and enable the AI Platform API
* In the navigation menu, choose <b>Storage/Storage</b>
* Create a new bucket called `custom-estimator-mlengine`
* Upload your model to the bucket

# Deploy models and versions

* In the navigation menu, choose <b>Artificial Intelligence/AI Platform</b>, then choose <b>Models</b>
* Select <b>Create model</b>. In the <b>New model</b> menu, specify a model name (e.g. `iris-tensorflow`) and a region, and then create the model.
* Select the newly created model, choose <b>New version</b>.
* In the <b>New version</b> menu:
  * Specify a version name. For example `v1`
  * Specify a python version
  * For framework, choose TensorFlow
  * Specify framework version and ML runtime version
  * For the model URI, enter the path to your SavedModel directory in Cloud Storage. This is often a timestamped directory like `gs://custom-estimator-mlengine/iris-tensorflow/export/Servo/1563629175/`
  * Choose <b>Save</b> to create a new version

# Use Postman to test the API

* Get bearer token for authorization

  https://cloud.google.com/vision/docs/auth#using_a_service_account
  
  * Open the Google Cloud SDK Shell
  * Authenticate to your service account, replacing `KEY_FILE` below with the path to your service account key file:
  
    `gcloud auth activate-service-account --key-file KEY_FILE`
  
  * Obtain an authorization token using your service account:
  
    `gcloud auth print-access-token`
  
* HTTP URL format:

  https://cloud.google.com/ml-engine/docs/v1/predict-request#http-url-format

  `POST https://ml.googleapis.com/v1/projects/my-project/models/my-model/versions/my-version:predict`

* Launch Postman:
 * For <b>Authorization</b>, choose <b>Bearer Token</b> and enter the token
 * For url, enter
 
   `https://ml.googleapis.com/v1/projects/custom-estimator/models/iris-tensorflow/versions/v1:predict` 
 
   and choose POST method
 * For message body, enter
 ```json
 {
    "instances": [
        {
            "SepalLength": 6.4,
            "SepalWidth": 2.8,
            "PetalLength": 5.6,
            "PetalWidth": 2.2
        },
        {
            "SepalLength": 4.9,
            "SepalWidth": 3.1,
            "PetalLength": 1.5,
            "PetalWidth": 0.1
        }
    ]
}
```

Result:

```json
{
    "predictions": [
        {
            "probabilities": [
                1.02038e-9,
                0.000515907,
                0.999484
            ],
            "classes": 2
        },
        {
            "probabilities": [
                0.999027,
                0.000972887,
                1.89823e-9
            ],
            "classes": 0
        }
    ]
}
```